# 11 - Apache Spark Básico: Data Processing y ETL

## 🎯 Objetivos
- Entender los fundamentos de Apache Spark
- Trabajar con DataFrames y transformaciones
- Realizar operaciones ETL (Extract, Transform, Load)
- SQL en Spark
- Agregaciones y joins
- Optimización básica

## 📚 Tecnologías
- **Apache Spark**: Motor de procesamiento distribuido
- **PySpark**: API de Python para Spark
- **Pandas**: Para comparaciones

## ⭐ Complejidad: Básico

## 1. Instalación y Setup

In [ ]:
# Instalar PySpark
!pip install pyspark pandas numpy matplotlib seaborn -q

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports completados")

## 2. Crear SparkSession

La SparkSession es el punto de entrada para trabajar con Spark.

In [ ]:
# Crear SparkSession
spark = SparkSession.builder \
    .appName("Spark Basico Tutorial") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

# Configurar nivel de log
spark.sparkContext.setLogLevel("ERROR")

print(f"✅ SparkSession creada")
print(f"📊 Spark version: {spark.version}")
print(f"📊 App name: {spark.sparkContext.appName}")
print(f"📊 Master: {spark.sparkContext.master}")

## 3. Crear DataFrames

Hay varias formas de crear DataFrames en Spark.

In [ ]:
# 1. Desde listas
data = [
    (1, "Juan", 28, "México", 50000),
    (2, "María", 35, "España", 75000),
    (3, "Pedro", 42, "Argentina", 60000),
    (4, "Ana", 29, "México", 55000),
    (5, "Luis", 38, "España", 70000)
]

columns = ["id", "nombre", "edad", "pais", "salario"]

df1 = spark.createDataFrame(data, columns)

print("📊 DataFrame desde lista:")
df1.show()

# 2. Desde Pandas
pandas_df = pd.DataFrame(data, columns=columns)
df2 = spark.createDataFrame(pandas_df)

print("\n📊 DataFrame desde Pandas:")
df2.show(3)

# 3. Con schema explícito
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("nombre", StringType(), True),
    StructField("edad", IntegerType(), True),
    StructField("pais", StringType(), True),
    StructField("salario", IntegerType(), True)
])

df3 = spark.createDataFrame(data, schema)

print("\n📊 Schema del DataFrame:")
df3.printSchema()

## 4. Operaciones Básicas con DataFrames

In [ ]:
# Usar df1 para ejemplos
df = df1

# Información básica
print("📊 Información del DataFrame:\n")
print(f"Número de filas: {df.count()}")
print(f"Número de columnas: {len(df.columns)}")
print(f"Columnas: {df.columns}")

# Primeras filas
print("\n📊 Primeras 3 filas:")
df.show(3)

# Describir datos
print("\n📊 Estadísticas descriptivas:")
df.describe().show()

# Tipos de datos
print("\n📊 Tipos de datos:")
df.printSchema()

## 5. Selección y Filtrado

In [ ]:
# Select - seleccionar columnas
print("📊 Select (nombre y salario):")
df.select("nombre", "salario").show()

# Select con expresiones
print("\n📊 Select con expresiones:")
df.select(
    "nombre",
    (F.col("salario") / 12).alias("salario_mensual")
).show()

# Filter - filtrar filas
print("\n📊 Filter (edad > 30):")
df.filter(F.col("edad") > 30).show()

# Múltiples condiciones
print("\n📊 Filter (edad > 30 AND pais = México):")
df.filter(
    (F.col("edad") > 30) & (F.col("pais") == "México")
).show()

# Where (sinónimo de filter)
print("\n📊 Where (salario >= 60000):")
df.where(F.col("salario") >= 60000).show()

## 6. Transformaciones y Nuevas Columnas

In [ ]:
# withColumn - agregar o modificar columna
print("📊 Agregar columna (salario_anual_bonus):")
df_with_bonus = df.withColumn(
    "salario_con_bonus",
    F.col("salario") * 1.10
)
df_with_bonus.show()

# Múltiples transformaciones
print("\n📊 Múltiples transformaciones:")
df_transformed = df \
    .withColumn("edad_grupo", 
                F.when(F.col("edad") < 30, "Joven")
                 .when(F.col("edad") < 40, "Adulto")
                 .otherwise("Senior")) \
    .withColumn("salario_k", F.round(F.col("salario") / 1000, 1)) \
    .withColumn("nombre_upper", F.upper(F.col("nombre")))

df_transformed.show()

# Renombrar columnas
print("\n📊 Renombrar columna:")
df.withColumnRenamed("pais", "country").show(3)

## 7. Agregaciones

Operaciones de agregación como sum, avg, max, min, count.

In [ ]:
# Agregaciones simples
print("📊 Agregaciones básicas:")
df.agg(
    F.count("*").alias("total"),
    F.avg("edad").alias("edad_promedio"),
    F.max("salario").alias("salario_max"),
    F.min("salario").alias("salario_min"),
    F.sum("salario").alias("salario_total")
).show()

# GroupBy
print("\n📊 GroupBy por país:")
df.groupBy("pais").agg(
    F.count("*").alias("empleados"),
    F.avg("edad").alias("edad_promedio"),
    F.avg("salario").alias("salario_promedio")
).show()

# Múltiples agrupaciones
print("\n📊 GroupBy con múltiples agregaciones:")
df_transformed.groupBy("pais", "edad_grupo").agg(
    F.count("*").alias("count"),
    F.round(F.avg("salario"), 2).alias("salario_avg")
).orderBy("pais", "edad_grupo").show()

## 8. Ordenamiento

In [ ]:
# OrderBy ascendente
print("📊 OrderBy (edad ascendente):")
df.orderBy("edad").show()

# OrderBy descendente
print("\n📊 OrderBy (salario descendente):")
df.orderBy(F.col("salario").desc()).show()

# Ordenar por múltiples columnas
print("\n📊 OrderBy múltiple (pais asc, salario desc):")
df.orderBy(
    F.col("pais").asc(),
    F.col("salario").desc()
).show()

## 9. SQL en Spark

Podemos usar SQL estándar en Spark DataFrames.

In [ ]:
# Registrar DataFrame como tabla temporal
df.createOrReplaceTempView("empleados")

# Query SQL básico
print("📊 SQL Query básico:")
spark.sql("""
    SELECT nombre, pais, salario
    FROM empleados
    WHERE salario > 60000
""").show()

# Query con agregación
print("\n📊 SQL con agregación:")
spark.sql("""
    SELECT 
        pais,
        COUNT(*) as empleados,
        ROUND(AVG(edad), 2) as edad_promedio,
        ROUND(AVG(salario), 2) as salario_promedio
    FROM empleados
    GROUP BY pais
    ORDER BY salario_promedio DESC
""").show()

# Query complejo con subconsultas
print("\n📊 SQL con subconsulta:")
spark.sql("""
    SELECT *
    FROM empleados
    WHERE salario > (
        SELECT AVG(salario) FROM empleados
    )
""").show()

## 10. Joins

Combinar DataFrames con diferentes tipos de joins.

In [ ]:
# Crear segundo DataFrame para joins
departamentos_data = [
    (1, "Tecnología", "Edificio A"),
    (2, "Ventas", "Edificio B"),
    (3, "Marketing", "Edificio C"),
    (4, "Recursos Humanos", "Edificio A"),
    (6, "Finanzas", "Edificio B")  # ID 6 no existe en empleados
]

df_dept = spark.createDataFrame(
    departamentos_data, 
    ["id", "departamento", "ubicacion"]
)

print("📊 DataFrame de departamentos:")
df_dept.show()

# Inner Join
print("\n📊 Inner Join:")
df.join(df_dept, "id", "inner").show()

# Left Join
print("\n📊 Left Join:")
df.join(df_dept, "id", "left").show()

# Right Join
print("\n📊 Right Join:")
df.join(df_dept, "id", "right").show()

# Full Outer Join
print("\n📊 Full Outer Join:")
df.join(df_dept, "id", "outer").show()

## 11. Window Functions

Funciones de ventana para análisis avanzados.

In [ ]:
# Definir ventana por país
window_pais = Window.partitionBy("pais").orderBy(F.col("salario").desc())

# Ranking dentro de cada país
print("📊 Ranking por país:")
df.withColumn(
    "rank",
    F.row_number().over(window_pais)
).show()

# Dense rank
print("\n📊 Dense Rank:")
df.withColumn(
    "dense_rank",
    F.dense_rank().over(window_pais)
).show()

# Lag y Lead (valores anteriores y siguientes)
print("\n📊 Lag y Lead:")
df.withColumn(
    "salario_anterior",
    F.lag("salario", 1).over(window_pais)
).withColumn(
    "salario_siguiente",
    F.lead("salario", 1).over(window_pais)
).show()

# Agregaciones con ventana
print("\n📊 Agregaciones con ventana:")
df.withColumn(
    "salario_avg_pais",
    F.round(F.avg("salario").over(Window.partitionBy("pais")), 2)
).withColumn(
    "diff_vs_avg",
    F.round(F.col("salario") - F.avg("salario").over(Window.partitionBy("pais")), 2)
).show()

## 12. Lectura y Escritura de Datos

In [ ]:
# Crear datos más grandes para demo
import random
from datetime import datetime, timedelta

# Generar datos de ventas sintéticos
ventas_data = []
productos = ["Laptop", "Mouse", "Teclado", "Monitor", "Audífonos"]
regiones = ["Norte", "Sur", "Este", "Oeste"]

start_date = datetime(2024, 1, 1)

for i in range(1000):
    fecha = start_date + timedelta(days=random.randint(0, 365))
    ventas_data.append((
        i + 1,
        fecha.strftime("%Y-%m-%d"),
        random.choice(productos),
        random.choice(regiones),
        random.randint(1, 10),  # cantidad
        round(random.uniform(100, 2000), 2)  # precio
    ))

df_ventas = spark.createDataFrame(
    ventas_data,
    ["id", "fecha", "producto", "region", "cantidad", "precio"]
)

print("📊 DataFrame de ventas (muestra):")
df_ventas.show(10)

# Escribir CSV
df_ventas.write.mode("overwrite").csv("ventas_csv", header=True)
print("\n✅ Guardado en CSV: ventas_csv/")

# Escribir Parquet (formato columnar eficiente)
df_ventas.write.mode("overwrite").parquet("ventas_parquet")
print("✅ Guardado en Parquet: ventas_parquet/")

# Escribir JSON
df_ventas.limit(100).write.mode("overwrite").json("ventas_json")
print("✅ Guardado en JSON: ventas_json/")

In [ ]:
# Leer archivos
print("📊 Leer CSV:")
df_from_csv = spark.read.csv("ventas_csv", header=True, inferSchema=True)
df_from_csv.printSchema()
df_from_csv.show(5)

print("\n📊 Leer Parquet:")
df_from_parquet = spark.read.parquet("ventas_parquet")
df_from_parquet.show(5)

print("\n📊 Leer JSON:")
df_from_json = spark.read.json("ventas_json")
df_from_json.show(5)

## 13. Análisis ETL Completo

Ejemplo de pipeline ETL completo.

In [ ]:
# EXTRACT - Leer datos
df_raw = spark.read.parquet("ventas_parquet")

print("📊 EXTRACT - Datos crudos:")
print(f"Total registros: {df_raw.count()}")
df_raw.show(5)

# TRANSFORM - Transformaciones
df_clean = df_raw \
    .withColumn("fecha", F.to_date("fecha")) \
    .withColumn("total", F.col("cantidad") * F.col("precio")) \
    .withColumn("año", F.year("fecha")) \
    .withColumn("mes", F.month("fecha")) \
    .withColumn("trimestre", F.quarter("fecha")) \
    .filter(F.col("cantidad") > 0) \
    .filter(F.col("precio") > 0)

print("\n📊 TRANSFORM - Datos limpios:")
df_clean.show(5)

# Análisis por producto
df_producto = df_clean.groupBy("producto").agg(
    F.count("*").alias("ventas_count"),
    F.sum("cantidad").alias("cantidad_total"),
    F.round(F.sum("total"), 2).alias("ingresos_total"),
    F.round(F.avg("precio"), 2).alias("precio_promedio")
).orderBy(F.col("ingresos_total").desc())

print("\n📊 Análisis por producto:")
df_producto.show()

# Análisis temporal
df_temporal = df_clean.groupBy("año", "mes").agg(
    F.count("*").alias("ventas"),
    F.round(F.sum("total"), 2).alias("ingresos")
).orderBy("año", "mes")

print("\n📊 Análisis temporal:")
df_temporal.show()

# Análisis por región
df_region = df_clean.groupBy("region", "producto").agg(
    F.sum("cantidad").alias("cantidad_total"),
    F.round(F.sum("total"), 2).alias("ingresos_total")
).orderBy("region", F.col("ingresos_total").desc())

print("\n📊 Análisis por región y producto:")
df_region.show()

# LOAD - Guardar resultados
df_producto.write.mode("overwrite").parquet("analytics/producto")
df_temporal.write.mode("overwrite").parquet("analytics/temporal")
df_region.write.mode("overwrite").parquet("analytics/region")

print("\n✅ LOAD - Datos guardados en analytics/")

## 14. Optimización y Mejores Prácticas

In [ ]:
# 1. Cache - para DataFrames usados múltiples veces
df_cached = df_clean.cache()

print("📊 DataFrame cacheado")
print(f"Registros: {df_cached.count()}")

# 2. Repartition - redistribuir datos
df_repartitioned = df_clean.repartition(4, "region")
print(f"\n📊 Particiones: {df_repartitioned.rdd.getNumPartitions()}")

# 3. Coalesce - reducir particiones (más eficiente que repartition)
df_coalesced = df_clean.coalesce(2)
print(f"📊 Particiones después de coalesce: {df_coalesced.rdd.getNumPartitions()}")

# 4. Explain - ver plan de ejecución
print("\n📊 Plan de ejecución:")
df_clean.groupBy("producto").agg(F.sum("total")).explain()

# 5. Broadcast join - para tablas pequeñas
from pyspark.sql.functions import broadcast

# Simular tabla pequeña de categorías
categorias_data = [
    ("Laptop", "Electrónica"),
    ("Mouse", "Accesorios"),
    ("Teclado", "Accesorios"),
    ("Monitor", "Electrónica"),
    ("Audífonos", "Accesorios")
]
df_categorias = spark.createDataFrame(categorias_data, ["producto", "categoria"])

# Join con broadcast
df_with_cat = df_clean.join(
    broadcast(df_categorias),
    "producto",
    "left"
)

print("\n📊 Join con broadcast:")
df_with_cat.show(5)

## 15. Conversión Spark ↔ Pandas

In [ ]:
# Spark a Pandas
pandas_df = df_producto.toPandas()

print("📊 DataFrame de Pandas:")
print(type(pandas_df))
print(pandas_df.head())

# Visualizar con pandas/matplotlib
plt.figure(figsize=(10, 6))
pandas_df.plot(x='producto', y='ingresos_total', kind='bar', legend=False)
plt.title('Ingresos por Producto')
plt.xlabel('Producto')
plt.ylabel('Ingresos Totales')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Pandas a Spark
spark_df = spark.createDataFrame(pandas_df)
print("\n📊 De vuelta a Spark:")
spark_df.show()

## 16. UDFs (User Defined Functions)

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, FloatType

# UDF simple
@udf(returnType=StringType())
def categorizar_precio(precio):
    if precio < 500:
        return "Bajo"
    elif precio < 1000:
        return "Medio"
    else:
        return "Alto"

# Aplicar UDF
df_with_category = df_clean.withColumn(
    "precio_categoria",
    categorizar_precio(F.col("precio"))
)

print("📊 DataFrame con UDF:")
df_with_category.select("producto", "precio", "precio_categoria").show(10)

# UDF con decorador completo
def calcular_descuento(precio, cantidad):
    if cantidad >= 5:
        return precio * 0.9  # 10% descuento
    return precio

udf_descuento = udf(calcular_descuento, FloatType())

df_with_discount = df_clean.withColumn(
    "precio_con_descuento",
    F.round(udf_descuento(F.col("precio"), F.col("cantidad")), 2)
)

print("\n📊 DataFrame con descuento:")
df_with_discount.select("producto", "cantidad", "precio", "precio_con_descuento").show(10)

## 17. Limpieza

In [ ]:
# Unpersist cached data
df_cached.unpersist()

print("✅ Cache liberado")

## 18. Resumen y Mejores Prácticas

### ✅ Conceptos Clave:
1. **SparkSession**: Punto de entrada para Spark
2. **DataFrames**: Estructura de datos distribuida
3. **Transformaciones**: Lazy evaluation (map, filter, select)
4. **Acciones**: Trigger execution (count, show, collect)
5. **SQL**: Queries estándar SQL
6. **Partitioning**: Distribución de datos
7. **Caching**: Optimización de performance

### 💡 Mejores Prácticas:
- ✅ Usa transformaciones en lugar de UDFs cuando sea posible (más rápido)
- ✅ Cache DataFrames que uses múltiples veces
- ✅ Usa broadcast joins para tablas pequeñas
- ✅ Particiona datos por columnas frecuentemente filtradas
- ✅ Prefiere Parquet sobre CSV para almacenamiento
- ✅ Usa `explain()` para entender el plan de ejecución
- ✅ Evita `collect()` en datasets grandes (trae todo a memoria)
- ✅ Usa SQL cuando sea más legible
- ✅ Repartition antes de escribir grandes volúmenes

### 🚀 Próximos Pasos:
- Spark Streaming para datos en tiempo real
- MLlib para machine learning distribuido
- GraphX para procesamiento de grafos
- Integración con S3, HDFS, Kafka
- Optimización avanzada con Catalyst Optimizer

In [ ]:
print("🎉 Tutorial de Spark Básico completado!")
print(f"\n📊 Resumen:")
print(f"   - DataFrames creados: 5+")
print(f"   - Operaciones aprendidas: 20+")
print(f"   - Archivos generados: ventas_csv/, ventas_parquet/, analytics/")
print(f"\n💡 Spark es ideal para:")
print(f"   - Procesamiento de grandes volúmenes de datos")
print(f"   - ETL distribuido")
print(f"   - Analytics en tiempo real")
print(f"   - Machine Learning a escala")

# Detener SparkSession
spark.stop()
print("\n✅ SparkSession detenida")